<a href="https://colab.research.google.com/github/vanessajen/FlatLay-Generator/blob/main/FlatLay_Generator_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ControlNet Flatten V1 - Chuẩn không lỗi

# --- Bờớc 0: Disable FLAX trước khi import diffusers ---
import os
os.environ["DIFFUSERS_FLAX_AVAILABLE"] = "0"

# --- Bờớc 1: Import thư viện ---
import torch
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from controlnet_aux import CannyDetector
from PIL import Image
import numpy as np

# --- Bờớc 2: Load model ControlNet ---
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16
)

pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet, safety_checker=None, torch_dtype=torch.float16
)

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_xformers_memory_efficient_attention()
pipe.to("cuda")

# --- Bờớc 3: Khai báo Canny Detector ---
canny = CannyDetector()

# --- Bờớc 4: Xử lý 1 ảnh mẫu ---

def process_flatlay(input_image_path, prompt, output_path):
    image = Image.open(input_image_path)
    image = image.convert("RGB")
    image = image.resize((512, 512))

    # Tính toán canny map
    canny_image = canny(image)

    # Generate ảnh
    generated_image = pipe(
        prompt,
        num_inference_steps=30,
        guidance_scale=9.0,
        image=canny_image
    ).images[0]

    # Lưu ảnh
    generated_image.save(output_path)

# --- Bờớc 5: Demo ---

input_image_path = "/content/drive/MyDrive/FlatLay_Input/sample.jpg"  # Chỉnh lại đường dẫn ảnh input
output_image_path = "/content/drive/MyDrive/FlatLay_Demo_Output/output_sample.jpg"
prompt = "flat lay minimal pastel background, photography, highly detailed, white background"

process_flatlay(input_image_path, prompt, output_image_path)

print("🌿✨ Done!")
